In [1]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
cd /content/gdrive/MyDrive/Univ of Surrey Work/sketch_adversarial-master

/content/gdrive/MyDrive/Univ of Surrey Work/sketch_adversarial-master


In [3]:
pip install bresenham  #line drawing algorithm

In [4]:
import argparse

In [5]:
from classifier.models import *
from classifier.dataset import get_dataloader

ModuleNotFoundError: ignored

In [6]:
cd /content/gdrive/MyDrive/Univ of Surrey Work/sketch_adversarial-master/classifier

/content/gdrive/MyDrive/Univ of Surrey Work/sketch_adversarial-master/classifier


In [7]:
from classifier.models import *
from classifier.dataset import get_dataloader

In [8]:
import numpy as np
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import torch.nn as nn
import torch.nn.functional as F #233
import torch.optim as optim
from torchvision import datasets,models,transforms
from PIL import Image

In [9]:
cd /content/gdrive/MyDrive/Univ of Surrey Work/sketch_adversarial-master/WhiteBoxAttacks/CW

/content/gdrive/MyDrive/Univ of Surrey Work/sketch_adversarial-master/WhiteBoxAttacks/CW


In [10]:
pip install texttable

In [11]:
from cw import CarliniWagner

In [12]:
import sys
sys.argv=['']
del sys

In [13]:
cd /content/gdrive/MyDrive/Univ of Surrey Work/sketch_adversarial-master/

/content/gdrive/MyDrive/Univ of Surrey Work/sketch_adversarial-master


In [15]:
if __name__ == "__main__":

    parser = argparse.ArgumentParser(description='Skecth_Classification')
    parser.add_argument('--backbone_name', type=str, default='Resnet', help='VGG / InceptionV3/ Resnet')
    parser.add_argument('--pool_method', type=str, default='AdaptiveAvgPool2d', help='AdaptiveMaxPool2d / AdaptiveAvgPool2d / AvgPool2d')
    parser.add_argument('--batchsize', type=int, default=64)
    parser.add_argument('--nThreads', type=int, default=8)
    parser.add_argument('--splitTrain', type=float, default=0.8)
    parser.add_argument('--learning_rate', type=float, default=0.0001)

    hp = parser.parse_args()
    dataloader_Train, dataloader_Test = get_dataloader(hp)
    print(hp)

    hp.FGSM_MNIST = {'classnum': 250, 'confidence': 1e-4, 'clip_max': 1, 'clip_min': 0, 'max_iterations': 1000, 'initial_const': 1e-2, 
                     'binary_search_steps': 5, 'learning_rate': 0.00001,'abort_early': True}

    model = Sketch_Classification(hp)
    model.to(device)
    model.load_state_dict(torch.load('./classifier/model_best_TUBerlin_Bina.pth', map_location=device))
    model.eval()

    with torch.no_grad():
        True_Accuracy = model.evaluate(dataloader_Test)

    print('True_Accuracy: {:.4f}'.format(True_Accuracy))

Total Training Sample 15999
Total Testing Sample 4000
Total Training Sample 15999
Total Testing Sample 4000
Namespace(backbone_name='Resnet', batchsize=64, learning_rate=0.0001, nThreads=8, pool_method='AdaptiveAvgPool2d', splitTrain=0.8)


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))



Test set: Average loss: 0.0042, Accuracy: 3729/4000 (93%), Time_Takes: 49.84748339653015

True_Accuracy: 93.2250


# New Section

In [ ]:
if __name__ == "__main__":

    parser = argparse.ArgumentParser(description='Skecth_Classification')
    parser.add_argument('--backbone_name', type=str, default='Resnet', help='VGG / InceptionV3/ Resnet')
    parser.add_argument('--pool_method', type=str, default='AdaptiveAvgPool2d', help='AdaptiveMaxPool2d / AdaptiveAvgPool2d / AvgPool2d')
    parser.add_argument('--batchsize', type=int, default=64)
    parser.add_argument('--nThreads', type=int, default=8)
    parser.add_argument('--splitTrain', type=float, default=0.8)
    parser.add_argument('--learning_rate', type=float, default=0.0001)

    hp = parser.parse_args()
    dataloader_Train, dataloader_Test = get_dataloader(hp)
    print(hp)

    hp.FGSM_MNIST = {'classnum': 250, 'confidence': 1e-4, 'clip_max': 1, 'clip_min': 0, 'max_iterations': 1000, 'initial_const': 1e-2, 
                     'binary_search_steps': 5, 'learning_rate': 0.00001,'abort_early': True}

    model = Sketch_Classification(hp)
    model.to(device)
    model.load_state_dict(torch.load('./classifier/model_best_TUBerlin_Bina.pth', map_location=device))
    model.eval()

    True_Accuracy = 93.2250

    attack = CarliniWagner(model, device='cuda')

    correct, correct_adv, correct_preserved = 0, 0, 0
    test_loss, test_loss_adv = 0, 0
    start_time = time.time()

    for i_batch, batch in enumerate(dataloader_Test):

        print(i_batch)
        images = batch['sketch_img'].to(device)
        images = (images > 0.4).float()

        for img, label in zip(images, batch['sketch_label']):
            target_label = torch.randint(0, 249, (1,))[0].to(device)
            img = img.unsqueeze(0)
            AdvExArray = attack.generate(img, label, target_label, **hp.FGSM_MNIST)
            # AdvExArray = AdvExArray.unsqueeze_(0).float()

            output = model(AdvExArray)
            test_loss_adv += model.loss(output, label.to(device).unsqueeze(0)).item()
            prediction_adv = output.argmax(dim=1, keepdim=True).to('cpu')
            correct_adv += prediction_adv.eq(label.view_as(prediction_adv)).sum().item()

            AdvExArray_Bina =  (AdvExArray > 0.4).float()
            save_image(torch.cat((img, AdvExArray, AdvExArray_Bina), dim=0), 'images.jpg')
            # print(torch.equal(AdvExArray_Bina, img))
            output = model(AdvExArray_Bina)
            test_loss += model.loss(output, label.to(device).unsqueeze(0)).item()
            prediction = output.argmax(dim=1, keepdim=True).to('cpu')
            correct_preserved += prediction.eq(label.view_as(prediction)).sum().item()

    #        Save images


    Adv_Accuracy = 100. * correct_adv / len(dataloader_Test.dataset)
    Adv_Accuracy_preserved = 100. * correct_preserved / len(dataloader_Test.dataset)
    print('True_Accuracy: {:.4f}, Adv_Accuracy, Adv_Accuracy_preserved: {},  Time_Takes: {}\n'.format(True_Accuracy,
                                                                                                      Adv_Accuracy,
                                                                                                      Adv_Accuracy_preserved,
                                                                                                      (time.time() - start_time)))
                                                                                                      


Total Training Sample 15999
Total Testing Sample 4000
Total Training Sample 15999
Total Testing Sample 4000
Namespace(backbone_name='Resnet', batchsize=64, learning_rate=0.0001, nThreads=8, pool_method='AdaptiveAvgPool2d', splitTrain=0.8)


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


0


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


iteration:0,loss:0.1487
iteration:0,loss:0.0780
iteration:0,loss:0.1555
iteration:0,loss:0.1418
iteration:0,loss:0.1204
iteration:0,loss:0.0370
iteration:0,loss:0.1274
iteration:0,loss:0.0914
iteration:0,loss:0.1724
iteration:0,loss:0.1491
iteration:0,loss:0.0831
iteration:0,loss:0.1092
iteration:0,loss:0.0620
iteration:0,loss:0.0912
iteration:0,loss:0.0686
iteration:0,loss:0.1321
iteration:0,loss:0.1179
iteration:0,loss:0.1083
iteration:0,loss:0.0877
iteration:0,loss:0.1351
iteration:0,loss:0.1269
iteration:0,loss:0.0841
iteration:0,loss:0.1466
iteration:0,loss:0.1398
iteration:0,loss:0.0971
iteration:0,loss:0.1357
iteration:0,loss:0.1345
iteration:0,loss:0.1325
iteration:0,loss:0.1452
iteration:0,loss:0.1077
iteration:0,loss:0.0999


In [ ]:
if __name__ == "__main__":

    parser = argparse.ArgumentParser(description='Skecth_Classification')
    parser.add_argument('--backbone_name', type=str, default='Resnet', help='VGG / InceptionV3/ Resnet')
    parser.add_argument('--pool_method', type=str, default='AdaptiveAvgPool2d', help='AdaptiveMaxPool2d / AdaptiveAvgPool2d / AvgPool2d')
    parser.add_argument('--batchsize', type=int, default=64)
    parser.add_argument('--nThreads', type=int, default=8)
    parser.add_argument('--splitTrain', type=float, default=0.8)
    parser.add_argument('--learning_rate', type=float, default=0.0001)

    hp = parser.parse_args()
    dataloader_Train, dataloader_Test = get_dataloader(hp)
    print(hp)

    hp.FGSM_MNIST = {'classnum': 10, 'confidence': 1e-4, 'clip_max': 1, 'clip_min': 0, 'max_iterations': 1000, 'initial_const': 1e-2, 
                     'binary_search_steps': 5, 'learning_rate': 0.00001,'abort_early': True}

    model = Sketch_Classification(hp)
    model.to(device)
    model.load_state_dict(torch.load('./classifier/model_best_TUBerlin_Bina.pth', map_location=device))
    model.eval()

    True_Accuracy = 93.2250

    attack = CarliniWagner(model, device='cuda')

    correct, correct_adv, correct_preserved = 0, 0, 0
    test_loss, test_loss_adv = 0, 0
    start_time = time.time()

    for i_batch, batch in enumerate(dataloader_Test):

        print(i_batch)
        images = batch['sketch_img'].to(device)
        images = (images > 0.4).float()

        for img, label in zip(images, batch['sketch_label']):
            target_label = torch.randint(0, 10, (1,))[0].to(device)
            img = img.unsqueeze(0)
            print(label)
            AdvExArray = attack.generate(img, label, target_label, **hp.FGSM_MNIST)
            # AdvExArray = AdvExArray.unsqueeze_(0).float()

            output = model(AdvExArray)
            test_loss_adv += model.loss(output, label.to(device).unsqueeze(0)).item()
            prediction_adv = output.argmax(dim=1, keepdim=True).to('cpu')
            correct_adv += prediction_adv.eq(label.view_as(prediction_adv)).sum().item()

            AdvExArray_Bina =  (AdvExArray > 0.4).float()
            save_image(torch.cat((img, AdvExArray, AdvExArray_Bina), dim=0), 'images.jpg')
            # print(torch.equal(AdvExArray_Bina, img))
            output = model(AdvExArray_Bina)
            test_loss += model.loss(output, label.to(device).unsqueeze(0)).item()
            prediction = output.argmax(dim=1, keepdim=True).to('cpu')
            correct_preserved += prediction.eq(label.view_as(prediction)).sum().item()

    #        Save images


    Adv_Accuracy = 100. * correct_adv / len(dataloader_Test.dataset)
    Adv_Accuracy_preserved = 100. * correct_preserved / len(dataloader_Test.dataset)
    print('True_Accuracy: {:.4f}, Adv_Accuracy, Adv_Accuracy_preserved: {},  Time_Takes: {}\n'.format(True_Accuracy,
                                                                                                      Adv_Accuracy,
                                                                                                      Adv_Accuracy_preserved,
                                                                                                      (time.time() - start_time)))

Total Training Sample 15999
Total Testing Sample 4000
Total Training Sample 15999
Total Testing Sample 4000
Namespace(backbone_name='Resnet', batchsize=64, learning_rate=0.0001, nThreads=8, pool_method='AdaptiveAvgPool2d', splitTrain=0.8)


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


0
tensor(0)


RuntimeError: ignored